# ODTC Tutorial: Connect, Lid, Block Temperature, and Protocol

This notebook walks through the ODTC (Inheco) thermocycler interface: setup, listing methods, lid (door) commands, setting block temperature, and running a protocol. Use it as a reference for the recommended workflow.

## 1. Imports and thermocycler

Use **ODTCThermocycler** (recommended): it owns `odtc_ip`, `variant`, and dimensions. Alternative: generic `Thermocycler` + `ODTCBackend` for custom backend options.

In [ ]:
import logging
from pylabrobot.resources import Coordinate
from pylabrobot.thermocycling.inheco import ODTCThermocycler
# Preferred: ODTCThermocycler (built-in ODTC dimensions; variant 96 or 384)
tc = ODTCThermocycler(name="odtc_test", odtc_ip="192.168.1.50", variant=96, child_location=Coordinate.zero())
# Override: tc = Thermocycler(..., backend=ODTCBackend(odtc_ip=..., variant=96, logger=...), ...) for custom backend

## 2. Connect and list device methods and premethods

`setup()` resets and initializes the device. **Methods** = runnable protocols on the device; **premethods** = setup-only (e.g. set temperature). Use **`tc.backend.list_protocols()`** to get a **`ProtocolList`** (`.methods`, `.premethods`, `.all`); **`tc.backend.get_protocol(name)`** returns an **ODTCProtocol** for methods (`None` for premethods).

In [ ]:
await tc.setup()
print("✓ Connected and initialized.")

**ODTCProtocol and running options:** **`tc.backend.get_protocol(name)`** returns an **ODTCProtocol** (subclasses `Protocol`; has `.steps`, `.name`, and ODTC-specific fields). Use **`print(odtc)`** for a human-readable summary.

- **Roundtrip:** **`tc.run_protocol(odtc, block_max_volume)`** — pass the ODTCProtocol from the device; same device-calculated config (thermal tuning preserved).
- **Run by name (recommended for PCR):** **`tc.run_stored_protocol("MethodName")`** — device runs its stored method; optimal thermal (overshoots and device-tuned ramps).
- **Custom protocol:** **`tc.run_protocol(protocol, block_max_volume, config=...)`** with a generic `Protocol` and optional **`tc.backend.get_default_config(post_heating=...)`** — no prior device config means default overshoot; use roundtrip or run-by-name for best thermal performance.

In [ ]:
protocol_list = await tc.backend.list_protocols()
# Print methods and premethods in clear sections
print(protocol_list)

# Iteration and .all still work: for name in protocol_list, protocol_list.all
# Optional: inspect a runnable method (get_protocol returns None for premethods)
# ODTCProtocol subclasses Protocol; print(odtc) shows full structure and steps
if protocol_list.all:
    first_name = protocol_list.methods[0] if protocol_list.methods else protocol_list.premethods[0]
    fetched_protocol = await tc.backend.get_protocol(first_name)
    if fetched_protocol is not None:
        print("\nExample stored protocol (full structure and steps):")
        print(fetched_protocol)
        # Roundtrip: run with same ODTC config via run_protocol(odtc, block_max_volume)


## 3. Lid (door) commands

The Thermocycler API uses **`open_lid`** / **`close_lid`** (ODTC device calls this the door). Door open/close use a 60 s estimated duration and corresponding timeout. Use **`wait=False`** to get an execution handle and avoid blocking; then **`await handle.wait()`** or **`await handle`** when you need to wait. For method runs, progress is logged every **progress_log_interval** (default 150 s) while you await. Omit `wait=False` to block until the command finishes.

**Waiting:** `await handle.wait()` or `await handle` (same). For method/protocol runs, **`await tc.wait_for_profile_completion(poll_interval=..., timeout=...)`** uses polling and supports a timeout.

In [ ]:
# Non-blocking: returns CommandExecution handle; await handle.wait() when you need to wait
door_handle = await tc.close_lid(wait=False)
print(f"Close started (request_id={door_handle.request_id})")

## 4. Block temperature and protocol

ODTC has no direct “set block temp” command; **`set_block_temperature`** uploads and runs a PreMethod. Use **`wait=False`** to get a handle; **`run_protocol(protocol, block_max_volume, config=...)`** is always non-blocking — await the returned handle or **`tc.wait_for_profile_completion()`** to block. When you **await** a method handle, progress (from DataEvents) is reported every **progress_log_interval** (default 150 s). Use **`config=tc.backend.get_default_config(post_heating=True)`** to hold temperatures after the method ends.

In [ ]:
await door_handle.wait()
print("Door closed.")

# set_block_temperature runs a premethod; wait=False returns MethodExecution handle
# Override: debug_xml=True, xml_output_path="out.xml" to save generated MethodSet XML
mount_handle = await tc.set_block_temperature([37.0],
                                              wait=False,
                                              debug_xml=True,
                                              xml_output_path="debug_set_mount_temp.xml"
                                              )
block = await tc.get_block_current_temperature()
lid = await tc.get_lid_current_temperature()
print(f"Block: {block[0]:.1f} °C  Lid: {lid[0]:.1f} °C")

In [ ]:
# Poll temps while method runs (optional)
block, lid = await tc.get_block_current_temperature(), await tc.get_lid_current_temperature()
print(f"Block: {block[0]:.1f} °C  Lid: {lid[0]:.1f} °C")

In [ ]:
from pylabrobot.thermocycling.standard import Protocol, Stage, Step

# Wait for set_block_temperature (previous cell) to finish before starting a protocol
await mount_handle

# run_protocol is always non-blocking; returns MethodExecution. To block: await handle.wait() or tc.wait_for_profile_completion()
config = tc.backend.get_default_config(post_heating=False)  # if True: hold temps after method ends
cycle_protocol = Protocol(stages=[
    Stage(steps=[
        Step(temperature=[37.0], hold_seconds=10.0),
        Step(temperature=[60.0], hold_seconds=10.0),
        Step(temperature=[10.0], hold_seconds=10.0),
    ], repeats=1)
])
execution = await tc.run_protocol(cycle_protocol, 50.0, config=config)
# Override: run_stored_protocol("MethodName") to run a device-stored method by name
print(f"Protocol started (request_id={execution.request_id})")
block, lid = await tc.get_block_current_temperature(), await tc.get_lid_current_temperature()
print(f"Block: {block[0]:.1f} °C  Lid: {lid[0]:.1f} °C")


**Status and progress during a protocol run** — When you **await execution** (or **await execution.wait()**), progress is logged every **progress_log_interval** (default 150 s) from the latest DataEvent (elapsed time, step/cycle, temperatures). You can also poll status: **`is_profile_running()`**, **`get_hold_time()`**, **`get_current_cycle_index()`** / **`get_total_cycle_count()`**, **`get_current_step_index()`** / **`get_total_step_count()`**. Run the cell below after starting the protocol to poll and print status manually.

In [ ]:
import asyncio

# Poll status a few times while the protocol runs (run this cell after starting the protocol)
for poll in range(6):
    running = await tc.is_profile_running()
    if not running:
        print(f"Poll {poll + 1}: profile no longer running.")
        break
    hold_s = await tc.get_hold_time()
    cycle = await tc.get_current_cycle_index()
    total_cycles = await tc.get_total_cycle_count()
    step = await tc.get_current_step_index()
    total_steps = await tc.get_total_step_count()
    block = await tc.get_block_current_temperature()
    lid = await tc.get_lid_current_temperature()
    print(f"Poll {poll + 1}: cycle {cycle + 1}/{total_cycles}, step {step + 1}/{total_steps}, hold_remaining={hold_s:.1f}s, block={block[0]:.1f}°C, lid={lid[0]:.1f}°C")
    await asyncio.sleep(10)
else:
    print("Still running after 6 polls; use await execution.wait() or tc.wait_for_profile_completion() to block until done.")

## 5. Wait, open lid, disconnect

Await protocol completion, open the lid (non-blocking then wait), then **`tc.stop()`** to close the connection.

In [ ]:
# Block until protocol done; progress is logged every 150 s (progress_log_interval) while waiting
# Alternatively: await tc.wait_for_profile_completion(poll_interval=..., timeout=...)
await execution.wait()

open_handle = await tc.open_lid(wait=False)
await open_handle.wait()
await tc.stop()
print("Done.")